In [1]:
import os
import numpy as np
import cv2
import pandas as pd
from sklearn.cluster import KMeans
from scipy.cluster.vq import vq
from hmmlearn import hmm

    


In [2]:
# Function to extract SIFT features from a single window
def extract_sift_features(window):
    if len(window.shape) == 3:
        window = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(window, None)
    return descriptors

# Function to quantize descriptors using the KMeans codebook
def quantize_descriptors(descriptors, kmeans):
    if descriptors is not None:
        words, _ = vq(descriptors, kmeans.cluster_centers_)
        return words
    return np.array([])

# Create histogram for quantized descriptors (BoF)
def create_bof_histogram(words, num_clusters):
    histogram, _ = np.histogram(words, bins=np.arange(num_clusters + 1), density=True)
    return histogram


In [3]:
# Paths and Parameters
image_folder = r"C:\Users\dhanu\Downloads\color_window_double1 (1) 5\color_window_double1 (1)\color_window_double1"  # Replace with your actual image folder path
excel_path = "gt_Window (2) 2 (1).xlsx" 
labels_df = pd.read_excel(excel_path)

window_width = 90
step_size = 20
num_clusters = 100

# Build the KMeans codebook
all_descriptors = []
for filename in os.listdir(image_folder):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
        image_path = os.path.join(image_folder, filename)
        image = cv2.imread(image_path)
        if image is not None:
            for j in range(0, image.shape[1] - window_width + 1, step_size):
                window = image[:, j:j + window_width]
                descriptors = extract_sift_features(window)
                if descriptors is not None:
                    all_descriptors.append(descriptors)

if all_descriptors:
    all_descriptors = np.vstack(all_descriptors)
    kmeans = KMeans(n_clusters=num_clusters, random_state=42)
    kmeans.fit(all_descriptors)
else:
    raise ValueError("No descriptors found. Check image paths and data.")


In [4]:
import os
import cv2
import numpy as np
import pandas as pd

# Parameters
image_folder = r"C:\Users\dhanu\Downloads\color_window_double1 (1) 5\color_window_double1 (1)\color_window_double1"  # Update with the actual path to your images folder
excel_path = "gt_Window (2) 2 (1).xlsx"  # Update with the actual path to your Excel file
window_width = 150
step_size = 20

# Load the Excel file
labels_df = pd.read_excel(excel_path)

# Ensure the SIFT feature extraction function is defined
def extract_sift_features(window):
    """Extract SIFT features from a given window."""
    if len(window.shape) == 3:
        window = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(window, None)
    return descriptors

# Collect all SIFT descriptors
all_descriptors = []
for _, row in labels_df.iterrows():
    image_name = row['image name']  # Column name for the image file
    image_path = os.path.join(image_folder, image_name)  # Construct full image path

    # Check if the image exists
    if not os.path.exists(image_path):
        print(f"Image {image_name} not found in {image_folder}, skipping...")
        continue

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Failed to load image: {image_name}, skipping...")
        continue

    # Perform sliding window and collect SIFT descriptors
    for j in range(0, image.shape[1] - window_width + 1, step_size):
        window = image[:, j:j + window_width]
        descriptors = extract_sift_features(window)
        if descriptors is not None:
            all_descriptors.append(descriptors)

# Flatten the list of descriptors into a single array
if all_descriptors:
    all_descriptors = np.vstack(all_descriptors)  # Stack all descriptors into one array
    print(f"Total descriptors collected: {all_descriptors.shape}")
else:
    raise ValueError("No descriptors found. Please check your dataset and paths.")


Total descriptors collected: (4773251, 128)


In [5]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(all_descriptors)  # Ensure all_descriptors is generated


KMeans(n_clusters=100, random_state=42)

In [6]:
import os
import cv2
import numpy as np
import pandas as pd

# Parameters
image_folder = r"C:\Users\dhanu\Downloads\color_window_double1 (1) 5\color_window_double1 (1)\color_window_double1"   # Update this to the actual folder containing your images
window_width =    150           # Width of the sliding window
step_size = 20                 # Step size for the sliding window
num_clusters = 100              # Number of clusters for KMeans

# Ensure the BoF generation functions are defined
def extract_sift_features(window):
    """Extract SIFT features from a given window."""
    if len(window.shape) == 3:
        window = cv2.cvtColor(window, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create()
    _, descriptors = sift.detectAndCompute(window, None)
    return descriptors

def quantize_descriptors(descriptors, kmeans):
    """Quantize descriptors using the KMeans codebook."""
    if descriptors is not None:
        from scipy.cluster.vq import vq
        words, _ = vq(descriptors, kmeans.cluster_centers_)
        return words
    return np.array([])

def create_bof_histogram(words, num_clusters):
    """Create a histogram for the quantized descriptors."""
    histogram, _ = np.histogram(words, bins=np.arange(num_clusters + 1), density=True)
    return histogram

# Load the labels file
excel_path = "gt_Window (2) 2 (1).xlsx"  # Update to the correct path
labels_df = pd.read_excel(excel_path)

# Assuming kmeans is already trained using descriptors
# Example: kmeans = KMeans(n_clusters=num_clusters, random_state=42)

# Generate BoF histograms
character_bof_features = {}
for _, row in labels_df.iterrows():
    image_name = row['image name']  # Column name for the image file
    character_label = row['Label']  # Column name for the character label
    image_path = os.path.join(image_folder, image_name)  # Construct full image path

    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Image {image_name} not found, skipping...")
        continue

    # Initialize the character in the dictionary if not already present
    if character_label not in character_bof_features:
        character_bof_features[character_label] = []

    # Perform sliding window and extract BoF features
    for j in range(0, image.shape[1] - window_width + 1, step_size):
        window = image[:, j:j + window_width]
        descriptors = extract_sift_features(window)
        words = quantize_descriptors(descriptors, kmeans)
        if len(words) > 0:
            histogram = create_bof_histogram(words, num_clusters)
            character_bof_features[character_label].append(histogram)

# Display a summary of the generated BoF histograms
print("Summary of character BoF features:")
for char, features in character_bof_features.items():
    print(f"Character '{char}': {len(features)} features")


Summary of character BoF features:
Character 'െ': 508 features
Character 'തെ': 117 features
Character 'അ': 151 features
Character 'ത്ത': 36 features
Character 'താ': 45 features
Character 'ി': 117 features
Character 'പ': 99 features
Character 'പറ': 27 features
Character 'രു': 72 features
Character 'രള': 9 features
Character 'ാളക': 9 features
Character 'ഷ': 9 features
Character 'മാ': 63 features
Character 'ത്തി': 27 features
Character 'ആ': 99 features
Character 'ത്ര': 18 features
Character 'െഠ': 27 features
Character 'ക': 130 features
Character 'ാൽ': 27 features
Character 'ൽ': 18 features
Character 'ർഇ': 9 features
Character 'ഇ': 27 features
Character 'ഇെ': 36 features
Character 'ാെ': 81 features
Character 'കെ': 135 features
Character 'ങ്ക': 45 features
Character 'നെ': 126 features
Character 'ന': 108 features
Character 'റെ': 90 features
Character 'ജെ': 63 features
Character 'കറ': 36 features
Character 'കട്ര': 9 features
Character 'ട്രി': 16 features
Character 'ട്രിട': 9 features
Characte

In [7]:
from hmmlearn import hmm
import numpy as np
from sklearn.preprocessing import StandardScaler

# Initialize parameters for HMM
character_models = {}
num_states = 5  # Number of states in the HMM
max_iterations = 2000  # Increased number of iterations
tolerance = 1e-4  # Lower tolerance for better convergence

# Train an HMM for each character
for char, occurrences in character_bof_features.items():
    try:
        # Validate and normalize occurrences
        valid_occurrences = [np.array(occ).reshape(-1, 1) for occ in occurrences if occ is not None and len(occ) > 0]
        if not valid_occurrences:
            raise ValueError(f"No valid occurrences found for character '{char}'")

        # Stack all valid occurrences into a single 2D array
        char_X = np.vstack(valid_occurrences)

        # Normalize features using StandardScaler
        scaler = StandardScaler()
        char_X = scaler.fit_transform(char_X)

        # Generate the lengths array for valid occurrences
        char_lengths = [len(occ) for occ in valid_occurrences]

        # Check if the lengths array matches the total rows in char_X
        if sum(char_lengths) != len(char_X):
            raise ValueError(
                f"Lengths array {char_lengths} doesn't sum to {len(char_X)} samples for character '{char}'"
            )

        # Initialize and configure the HMM
        model = hmm.GaussianHMM(
            n_components=num_states,
            covariance_type="diag",
            n_iter=max_iterations,
            tol=tolerance,
            init_params=""  # Prevent overwriting startprob_ and transmat_
        )
        model.startprob_ = np.full(num_states, 1.0 / num_states)  # Uniform start probabilities
        model.transmat_ = np.full((num_states, num_states), 1.0 / num_states)  # Uniform transition probabilities

        # Train the HMM
        model.fit(char_X, char_lengths)
        character_models[char] = model  # Save the trained model for this character
        print(f"Model trained for character '{char}'")

    except ValueError as ve:
        print(f"Error training HMM for character '{char}': {ve}")
    except Exception as e:
        print(f"Unexpected error for character '{char}': {e}")

print("HMM models trained for all characters.")


Model trained for character 'െ'
Model trained for character 'തെ'
Model trained for character 'അ'
Model trained for character 'ത്ത'
Model trained for character 'താ'
Model trained for character 'ി'
Model trained for character 'പ'


Model is not converging.  Current: 6097.019214016074 is not greater than 6097.019317241014. Delta is -0.00010322493926651077


Model trained for character 'പറ'
Model trained for character 'രു'
Model trained for character 'രള'
Model trained for character 'ാളക'
Model trained for character 'ഷ'
Model trained for character 'മാ'
Model trained for character 'ത്തി'
Model trained for character 'ആ'
Model trained for character 'ത്ര'
Model trained for character 'െഠ'
Model trained for character 'ക'
Model trained for character 'ാൽ'
Model trained for character 'ൽ'
Model trained for character 'ർഇ'
Model trained for character 'ഇ'
Model trained for character 'ഇെ'
Model trained for character 'ാെ'
Model trained for character 'കെ'
Model trained for character 'ങ്ക'
Model trained for character 'നെ'
Model trained for character 'ന'
Model trained for character 'റെ'
Model trained for character 'ജെ'
Model trained for character 'കറ'
Model trained for character 'കട്ര'
Model trained for character 'ട്രി'
Model trained for character 'ട്രിട'
Model trained for character 'ട്ടു'
Model trained for character 'ട്ടുഠ'
Model trained for character 'ുഠ'

Model is not converging.  Current: 645272.1974707337 is not greater than 645272.197572752. Delta is -0.00010201835539191961


Model trained for character '""'
Model trained for character 'ധിച്ച'


...

Model trained for character 'ന്തും'
Model trained for character 'ണ്ടമ'
Model trained for character 'റവിറ'
Model trained for character 'വിക്കി'
Model trained for character 'ിക്കിര'
Model trained for character 'തിരൻ'
Model trained for character 'രധ'
Model trained for character 'രൺ'
Model trained for character 'ൺച'
Model trained for character 'ചാ'
Model trained for character 'ൃാല'
Model trained for character 'ലപ'
Model trained for character 'ൃഠപ'
Model trained for character 'ൃത്ത'
Model trained for character 'വൻ'
Model trained for character 'ൃൺാ'
Model trained for character 'ൻചാ'
Model trained for character 'ചാം'
Model trained for character 'ൃപെ'
Model trained for character 'ലരാ'
Model trained for character 'ലന'
Model trained for character 'റടക'
Model trained for character 'ടക്ക'
Model trained for character 'ക്കാറ'
Model trained for character 'ഹത'
Model trained for character 'െൽ'
Model trained for character 'റാക്ക'
Model trained for character 'ാക്കി'
Model trained for character 'തിര'
Mode

Model is not converging.  Current: 596.9358149182982 is not greater than 596.9359466941297. Delta is -0.0001317758315053652


Model trained for character 'ങ്ങള'
Model trained for character 'ത്രളും'
Model trained for character 'ളംഅ'
Model trained for character 'ാഅം'
Model trained for character 'അവി'
Model trained for character 'റടെക്ക'
Model trained for character 'ടക്കും'
Model trained for character 'ക്കുത'
Model trained for character 'റന്നെ'
Model trained for character 'ന്നെെ'
Model trained for character 'നെഠ'
Model trained for character 'കുന്ന'
Model trained for character 'ദന്നു'
Model trained for character 'നുറെ'
Model trained for character 'വന്ന'
Model trained for character 'ൃന്നത'
Model trained for character 'നതട്ര'
Model trained for character 'പ്രാക്ക'
Model trained for character 'െൾനു'
Model trained for character 'ൾഅ'
Model trained for character 'അരു'
Model trained for character 'രുളി'
Model trained for character 'ദളിെ'
Model trained for character 'ളിൈ'
Model trained for character 'ച്ചെ'
Model trained for character 'ച്ഛ'
Model trained for character 'യൂ'
Model trained for character 'യൂഅ'
Model trained fo

Model is not converging.  Current: 1441.4299527089374 is not greater than 1441.4334910243706. Delta is -0.0035383154331611877


Model trained for character 'ഴപഗ'
Model trained for character 'പശു'
Model trained for character 'ശുച'
Model trained for character 'ചത്ത'
Model trained for character 'ത്തകീ'
Model trained for character 'തകി്ട'
Model trained for character 'കീടക'
Model trained for character 'ന്നുഅ'
Model trained for character 'ത്രഅ'
Model trained for character 'അപ്പ'
Model trained for character 'ഠപ്പ'
Model trained for character 'പ്പശു'
Model trained for character 'ശുക്ക'
Model trained for character 'ളെെ'
Model trained for character 'ളടെ'
Model trained for character 'ടെന'
Model trained for character 'ടസമ'
Model trained for character 'സമീം'
Model trained for character 'മീലം'
Model trained for character 'ത്തപ'
Model trained for character 'തപർ'
Model trained for character 'പതിം'
Model trained for character 'തിന്ന'
Model trained for character 'ാലക'
Model trained for character 'ഴുക്ക'
Model trained for character 'ഴുക്കറ'
Model trained for character 'ൾഇ'
Model trained for character 'ഇത'
Model trained for charac

In [8]:
from hmmlearn import hmm
import numpy as np

# Example: Initialize character_hmms with Malayalam characters
character_hmms = {
    'ണ': hmm.GaussianHMM(n_components=3),
    'ൽ': hmm.GaussianHMM(n_components=3),
    'ു': hmm.GaussianHMM(n_components=3),
    'ഥ': hmm.GaussianHMM(n_components=3),
    'ൃ': hmm.GaussianHMM(n_components=3),
    'ബ': hmm.GaussianHMM(n_components=3),
    'ൂ': hmm.GaussianHMM(n_components=3),
    'ഃ': hmm.GaussianHMM(n_components=3),
    'ൻ': hmm.GaussianHMM(n_components=3),
    'അ': hmm.GaussianHMM(n_components=3),  # Adding extra characters for demonstration
    'ആ': hmm.GaussianHMM(n_components=3),
}

# Configure dummy HMM models (for testing purposes)
for model in character_hmms.values():
    model.startprob_ = np.array([1.0, 0.0, 0.0])
    model.transmat_ = np.array([[0.7, 0.3, 0.0], [0.0, 0.7, 0.3], [0.0, 0.0, 1.0]])
    model.means_ = np.array([[0.0], [3.0], [5.0]])
    model.covars_ = np.array([[1.0], [1.0], [1.0]])

# List of Malayalam characters to remove
characters_to_remove = ['ണ', ' ൾ', 'ു', 'ഥ', 'ൃ', 'ബ', 'ൂ', 'ഃ', 'ൻ']

# Remove characters if they exist in the dictionary
for char in characters_to_remove:
    print(char)
    if char in character_hmms:
        del character_hmms[char]
        print(f"Deleted HMM model for character: {char}")
    else:
        print(f"Character {char} not found in character_hmms.")

# Check remaining characters in character_hmms (optional)
print("Remaining characters in character_hmms:")
print(character_hmms.keys())


ണ
Deleted HMM model for character: ണ
 ൾ
Character  ൾ not found in character_hmms.
ു
Deleted HMM model for character: ു
ഥ
Deleted HMM model for character: ഥ
ൃ
Deleted HMM model for character: ൃ
ബ
Deleted HMM model for character: ബ
ൂ
Deleted HMM model for character: ൂ
ഃ
Deleted HMM model for character: ഃ
ൻ
Deleted HMM model for character: ൻ
Remaining characters in character_hmms:
dict_keys(['ൽ', 'അ', 'ആ'])


In [ ]:
# Dictionary to store actual vs predicted characters
results = {'Actual': [], 'Predicted': []}

# Process each test character and its sequences
for actual_char, test_sequences in character_bof_features.items():
    for test_sequence in test_sequences:
        test_sequence = np.array(test_sequence)
        
        # Dictionary to store scores for each character model
        scores = {}
        
        # Score the test sequence against each character's HMM model
        for char, model in character_models.items():
            try:
                scores[char] = model.score(test_sequence)
            except:
                scores[char] = -np.inf  # If model can't score, assign a low score
        
        # Predict the character with the highest score
        predicted_char = max(scores, key=scores.get)
        
        # Store the result
        results['Actual'].append(actual_char)
        results['Predicted'].append(predicted_char)